In [ ]:
import os
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

from IPython.display import display, HTML

# filenames: ./logs/{topo}/{cc}/{0Mbps-0Mbps}/{0ms-0ms}/{rep}_{hostname}.txt
base_path = 'logs'
dir_path = './' + base_path + '/{}/{}/{}Mbps-{}Mbps/{}ms-{}ms/{}-'

def read_flow(sender_file_name, receiver_file_name):
    s = pd.read_csv(sender_file_name, sep='\t')
    r = pd.read_csv(receiver_file_name, sep='\t')
    # print(sender_file_name)
    df = pd.merge(s, r, how='outer', on='pkt_id')
    df['latency [s]'] = df['rcv_t [s]'] - df['snd_t [s]']
    df['latency [ms]'] = (df['rcv_t [s]'] - df['snd_t [s]']) * 1000
    df['sec'] = df['rcv_t [s]'] - df.at[(0, 'snd_t [s]')]
    df['disk_sec'] = df['sec'].apply(np.floor)
    df['disk_msec'] = df['sec'].apply(lambda x: np.floor(x * 1000))
    return df

def tp_array(df, bucket_size_ms=100):
    m = int(df['disk_msec'].max() / bucket_size_ms)
    byt, borders, _ = stats.binned_statistic(df['disk_msec'], df['payload [bytes]'], 'sum', bins=[i*bucket_size_ms for i in range(m)])
    ndf = pd.DataFrame({'msec': borders[:-1], 'tp [Mbps]': byt * 0.008 / bucket_size_ms})
    return ndf

def mean_tp(df, cutoff_s=2):
    latest_time = df['disk_msec'].max()
    df = tp_array(df)
    df = df[(df['msec'] >= cutoff_s*1e3) & (df['msec'] < latest_time - cutoff_s*1e3)]
    return df['tp [Mbps]'].mean()

def mean_latency(df, cutoff_s=2):
    latest_time = df['disk_msec'].max()
    df = df[(df['disk_msec'] > cutoff_s * 1000) & (df['disk_msec'] < latest_time - cutoff_s * 1000)]
    return df['latency [ms]'].mean()


topologies = ['mptcp-host-pair', 'MPflow_lpkt','MPvsSP', 'MPvsSP_lpkt']
cc_algorithms = ['lia', 'olia', 'balia', 'wvegas']

In [ ]:
# logs/mptcp-host-pair/balia/1ms-1ms/0-h1.txt

topo = 'mptcp-host-pair'

tmp_dir_path = './' + base_path + '/{}/{}/{}ms-{}ms/{}-'

df = pd.DataFrame([])

delays = np.arange(1, 101, 20)
tp = 10

for cc in cc_algorithms:
    for delay_a in delays[:3]:
        for delay_b in [d for d in delays if d >= delay_a]:
            temp_df = pd.DataFrame()
            for rep in range(3):
                # file name without host specified!
                # file_name = dir_path.format(topo, cc, tp, tp, d_a, d_b, rep)
                file_name = tmp_dir_path.format(topo, cc, delay_a, delay_b, rep)
                flow = read_flow(file_name + 'h1.txt', file_name + 'h2.txt')
                temp_df = temp_df.append({
                    'cc': cc,
                    'd_a': delay_a,
                    'd_b': delay_b,
                    'rep': rep,
                    'mean_tp': mean_tp(flow),
                    'mean_de': mean_latency(flow)
                }, ignore_index=True)
                # print(flow)
            df = df.append({
                'cc': cc,
                'd_a': delay_a,
                'd_b': delay_b,
                'mean_tp': temp_df['mean_tp'].mean(),
                'std_tp': temp_df['mean_tp'].std(),
                'mean_de': temp_df['mean_de'].mean(),
                'std_de': temp_df['mean_de'].std(),
            }, ignore_index=True)

df

In [ ]:
x = df[df['d_a'] == 1]


plt.clf()
plt.ylim((0, 20))
plt.grid()
plt.title('Throughput with one delay fixed')
axis = plt.gca()

for cc in ['lia', 'olia', 'balia', 'wvegas']:
    tmp = x[x['cc'] == cc]
    tmp.plot(x='d_b', y='mean_tp', label=cc, ax=axis)
plt.legend()

# Changing Throughput Ananlysis

In [ ]:
tps_a = [5, 10, 15]
tps_b = [5, 10, 15]

df = pd.DataFrame()
delay = 10
topo = 'mp-vs-sp'

def load_experiments(topology, cc_algorithms, tps_a, tps_b, delays_a, delays_b, repetitions=3, postfix=''):
    df = pd.DataFrame()
    for cc in cc_algorithms:
        for tp_a in tps_a:
            for tp_b in tps_b:
                for delay_a in delays_a:
                    for delay_b in delays_b:
                        temp = pd.DataFrame()
                        for rep in range(repetitions):
                            # file name without host specified!
                            file_name = dir_path.format(topology, cc, tp_a, tp_b, delay_a, delay_b, rep)
                            flow1 = read_flow(file_name + 'h1{}.txt'.format(postfix), file_name + 'h2{}.txt'.format(postfix))
                            flow2 = read_flow(file_name + 'h3{}.txt'.format(postfix), file_name + 'h4{}.txt'.format(postfix))
                            temp = temp.append({
                                'cc': cc,
                                'tp_a': tp_a,
                                'tp_b': tp_b,
                                'delay_a': delay_a,
                                'delay_b': delay_b,
                                'rep': rep,
                                'mean_tp_flow1': mean_tp(flow1),
                                'mean_de_flow1': mean_latency(flow1),
                                'mean_tp_flow2': mean_tp(flow2),
                                'mean_de_flow2': mean_latency(flow2)
                            }, ignore_index=True)
                        df = df.append({
                            'cc': cc,
                            'tp_a': tp_a,
                            'tp_b': tp_b,
                            'mean_tp_flow1': temp['mean_tp_flow1'].mean(),
                            'std_tp_flow1': temp['mean_tp_flow1'].std(),
                            'mean_de_flow1': temp['mean_de_flow1'].mean(),
                            'std_de_flow1': temp['mean_de_flow1'].std(),
                            'mean_tp_flow2': temp['mean_tp_flow2'].mean(),
                            'std_tp_flow2': temp['mean_tp_flow2'].std(),
                            'mean_de_flow2': temp['mean_de_flow2'].mean(),
                            'std_de_flow2': temp['mean_de_flow2'].std()
                        }, ignore_index=True)
    return df

"""
for cc in cc_algorithms:
    for tp_a in tps_a:
        for tp_b in tps_b:
            temp = pd.DataFrame()
            for rep in range(3):
                # file name without host specified!
                file_name = dir_path.format(topo, cc, tp_a, tp_b, delay, delay, rep)
                flow1 = read_flow(file_name + 'h1.txt', file_name + 'h2.txt')
                flow2 = read_flow(file_name + 'h3.txt', file_name + 'h4.txt')
                temp = temp.append({
                    'cc': cc,
                    'tp_a': tp_a,
                    'tp_b': tp_b,
                    'rep': rep,
                    'mean_tp_flow1': mean_tp(flow1),
                    'mean_de_flow1': mean_latency(flow1),
                    'mean_tp_flow2': mean_tp(flow2),
                    'mean_de_flow2': mean_latency(flow2)
                }, ignore_index=True)
                # print(flow)
                #break
            df = df.append({
                'cc': cc,
                'tp_a': tp_a,
                'tp_b': tp_b,
                'mean_tp_flow1': temp['mean_tp_flow1'].mean(),
                'std_tp_flow1': temp['mean_tp_flow1'].std(),
                'mean_de_flow1': temp['mean_de_flow1'].mean(),
                'std_de_flow1': temp['mean_de_flow1'].std(),
                'mean_tp_flow2': temp['mean_tp_flow2'].mean(),
                'std_tp_flow2': temp['mean_tp_flow2'].std(),
                'mean_de_flow2': temp['mean_de_flow2'].mean(),
                'std_de_flow2': temp['mean_de_flow2'].std()
            }, ignore_index=True)
             
"""
df = load_experiments(topo, cc_algorithms, tps_a, tps_b, [delay], [delay])
df

In [ ]:
x = df[df['tp_a'] == 10]

fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(15, 7.5))

for i, cc in enumerate(cc_algorithms[:2]):
    for j, flow in enumerate([1, 2]):
        x[x['cc'] == cc].plot(x='tp_b', y='mean_tp_flow{}'.format(flow), label='flow {}'.format(flow), ax=axes[i,0])

# for cc in ['lia', 'olia', 'balia', 'wvegas'][:1]:
#     x[cc].plot(x='tp_b', y='mean_tp_flow1', legend=True, label=cc, ylim=(0, 20), grid=True, title='Throughput by cc')